In [2]:
from bs4 import BeautifulSoup as bs
import requests
import re
import json
import urllib.parse
import csv
import pickle
import pandas as pd
import numpy as np

In [2]:
api_key = ""

In [3]:
def get_goedata(address, api_key):
    
    address = urllib.parse.quote(address)
    url = "https://maps.googleapis.com/maps/api/geocode/json?address={city}&key={key}".format(city=address, key=api_key)
    city = ""
    canton = ""
    country = ""
    lat = 0
    lng = 0
    
    data = requests.get(url)
    geodata = json.loads(data.text)
    if geodata['status'] == "OK":
        for level in geodata["results"][0]["address_components"]:
            if "locality" in level["types"]:
                city = level['short_name']
            if 'administrative_area_level_1' in level["types"]:
                canton = level['short_name']
            if 'country' in level["types"]:
                country = level['short_name']
        lat = geodata["results"][0]['geometry']['location']['lat']
        lng = geodata["results"][0]['geometry']['location']['lng']

    return (city, canton, country, lat, lng)
    
    

In [3]:
df = pd.read_csv("../data/marathon-data.csv" ,sep='\t')

FileNotFoundError: File b'../data/marathon-data.csv' does not exist

In [30]:
df.head(50)

,Unnamed: 0,an,catégorie,doss,lieu,nom,race-year,rang,retard,temps
0,0,50,SD4,(3001),GB-Luton,Abbitt Pamela,2002,66.,"55.32,7","2:30.44,4"
1,1,52,SH4,(3002),GB-Luton,Abbitt Philip,2002,182.,"40.25,0","2:02.23,2"
2,2,44,RH,(9411),Renens VD,Abderhalden Seth,2002,248.,"52.40,0","1:29.58,1"
3,3,60,MH3,(101),Vernier,Abdulaziz Sirwan,2002,83.,"47.01,7","3:13.32,2"
4,4,49,MH4,(102),JPN-Ibaraki 310-0066,Abe Shinya,2002,222.,"2:56.38,8","5:39.21,8"
5,5,77,MD1,(33),ETH-Ethiopie,Abosa Emebet,2002,3.,"20.33,8","2:49.37,0"
6,6,46,RH,(9001),F-Pontarlier,Abraham Michel,2002,99.,"14.54,0","52.12,1"
7,7,47,RH,(9003),Studen BE,Abt Hans,2002,39.,"9.15,1","46.33,2"
8,8,69,SH2,(3004),Belmont-Lausanne,Accetta-Parisi Vicenzo,2002,340.,"37.04,5","1:45.12,4"
9,9,68,SD2,(3005),St-Saphorin-Morges,Aceti Vendrani Daniela,2002,137.,"40.55,4","2:01.49,9"


In [12]:
df = df.replace(np.nan, '', regex=True)

In [26]:
def translate_geodata(city_list, reset_idx=False):
    city_dict = {}
    idx = None
    try:
        print('Trying to load data from pickle')
        if reset_idx:
            idx = 0
            pickle.dump(0, open('../data/last_index.p', 'wb'))
        else:
            idx = pickle.load(open('../data/last_index.p', 'rb'))
    except (OSError, IOError) as e:
        print("load data error")

    print('Starting translating from {}'.format(idx))
    try:
        city_list = city_list[idx: ]
        for city in city_list:
            if len(city):
                city_dict[city] = get_goedata(city, api_key)
            idx += 1
            if idx % 1000 == 0:
                print(idx)
        else:
            print("Finish at {}".format(idx))
            update_dict(city_dict,idx)
            
    except Exception as e:
        print(e)
        print(idx)
        update_dict(city_dict,idx)

def update_dict(city_dict,idx):
    old_dict = pickle.load(open('../data/city_geo.p', 'rb'))
    city_dict.update(old_dict)
    pickle.dump(city_dict, open('../data/city_geo.p', 'wb'))
    pickle.dump(idx, open('../data/last_index.p', 'wb'))

In [14]:
city_list = list(df['lieu'].unique())

In [27]:
translate_geodata(city_list[:6001], False)

Trying to load data from pickle
Starting translating from 5999
6000
Finish at 6001


In [58]:
city_dicts = pickle.load(open('../data/city_geo.p', 'rb'))
city_dicts[''] = ('', '', '', 0, 0)
len(city_dicts.keys())

12016

In [68]:
df['format_city'] = df['lieu'].apply(lambda x :  city_dicts[str(x)][0])
df['Canton'] = df['lieu'].apply(lambda x :  city_dicts[str(x)][1])
df['country'] = df['lieu'].apply(lambda x :  city_dicts[str(x)][2])
df['lat'] = df['lieu'].apply(lambda x :  city_dicts[str(x)][3])
df['lng'] = df['lieu'].apply(lambda x :  city_dicts[str(x)][4])

In [1]:
df.head(10)

NameError: name 'df' is not defined

In [71]:
df.to_csv("../data/marathon-data-with-geo.csv" ,sep=',')